In [147]:
# 输出每个cell的运行时间
%load_ext autotime
# https://github.com/cpcloud/ipython-autotime
import warnings
warnings.filterwarnings("ignore")

In [148]:
import pickle
import keras
import keras_contrib
import pickle
import numpy as np

model_dic = "test_data_model"
word2idx = pickle.load(open('test_data_model/word2idx.pkl','rb'))
emb_matrix = pickle.load(open('test_data_model/emb_matrix.pkl','rb'))
model = keras.models.load_model("model_file/bi_lstm_crf_12_3_0_42.h5",
                                custom_objects={"CRF": keras_contrib.layers.CRF, "crf_loss": keras_contrib.losses.crf_loss,
                                                "crf_viterbi_accuracy": keras_contrib.metrics.crf_viterbi_accuracy})
BATCH_SIZE = 16
MAX_LEN = 70
PAD_LEN = 10

def prepare(sent):
    # Step1--滑动窗切分句子:
    #        上一个句子的pad_len + 当前窗 + 下一个句子的pad_len
    sent = [c for c in sent]
    if len(sent)//MAX_LEN < 1:
        sent = [PAD_LEN*["_padding"] + sent + (MAX_LEN - len(sent) + PAD_LEN)*["_padding"]]
    else:
        cut = []
        for i in range(int(len(sent)//MAX_LEN)+1):
            cur_sent = sent[i*MAX_LEN : (i+1)*MAX_LEN]
            if i == 0:
                # 开始
                cur_sent = PAD_LEN*["_padding"] + cur_sent + sent[(i+1)*MAX_LEN:(i+1)*MAX_LEN + PAD_LEN]
            elif i == int(len(sent)//MAX_LEN):
                # 结束
                if len(cur_sent) > 0:
                    cur_sent = sent[i*MAX_LEN - PAD_LEN : i*MAX_LEN]  + cur_sent + (MAX_LEN - len(cur_sent) + PAD_LEN)*["_padding"]
                else:
                    break
            else:
                # 中间
                cur_sent = sent[i*MAX_LEN - PAD_LEN : i*MAX_LEN] + cur_sent + sent[(i+1)*MAX_LEN:(i+1)*MAX_LEN + PAD_LEN]
                if len(cur_sent) < MAX_LEN + 2*PAD_LEN:
                    cur_sent = cur_sent + (MAX_LEN + 2*PAD_LEN - len(cur_sent))*["_padding"]
            cut.append(cur_sent)
        sent = cut
    
    # Step2--sent2idx:
    #        上一个句子的pad_len + 当前窗 + 下一个句子的pad_len
    for idx,s in enumerate(sent):
        num_list = []
        for c in s:
            if word2idx.get(c) != None:
                num_list.append(word2idx.get(str(c)))
            elif word2idx.get(c) == None:
                 num_list.append(word2idx.get("_unk"))
        sent[idx] = num_list
    return np.array(sent)

def predict(sent2idx):
    return model.predict(sent2idx, batch_size=BATCH_SIZE, verbose=True)

time: 11.8 s


In [210]:
sent = """
糖尿病下肢动脉病变多数情况下是全身动脉病变的一部
分,控制动脉硬化的多重危险因素是防治介入治疗后再狭窄的
基础。 糖尿病膝下动脉较膝上动脉更易于发生再狭窄,介入手
术时血管病变越严重,术后再狭窄风险越高,长期糖尿病代谢
异常和 Fg 升高可能是导致再狭窄的原因。
"""
len(sent)

133

time: 3 ms


In [143]:
sent_pre = prepare(sent)
for s in sent_pre:
    print(s)

[   0    0    0    0    0    0    0    0    0    0    3   27   50   33
  170  726  163  264   33  175  167  160  361  412  170  108  272  396
  163  264   33  175   14   95  354    3   57    4  275  121  163  264
  536  100   14  167   98  350  212   88   46  108  507  109  589  291
  109  106  111  682 1000  989   14    3  130  467   21    2   27   50
   33 1518  170  163  264  165 1518  150  163  264  324  684   99   69
   85  682 1000  989    4  589]
[  33 1518  170  163  264  165 1518  150  163  264  324  684   99   69
   85  682 1000  989    4  589  291  504    3  331  141   37  153   33
  175  654  454   98    4  331  111  682 1000  989  256  212  654   73
    4  269  145   27   50   33  168  197    3  176  123   58    2   84
   36    2  233   73   93   87  108  232  259  682 1000  989   14  336
   88   21    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [144]:
result = predict(prepare(sent))

2/2 [==============================] - 2s 798ms/step


In [151]:
len(result[0][0])

16

time: 3 ms


In [216]:
merge_result = []
for r,s in zip(result,sent_pre):
    r = r[10:-10].tolist()
    s = s[10:-10].tolist()
    r_not_pad = []
    for idx,each in enumerate(s):
        if each != 0:
            r_not_pad.append(r[idx])
    merge_result += r_not_pad
len(merge_result)

133

time: 3 ms


In [212]:
a = [1,2,3,4]
a.pop(1)
a

[1, 3, 4]

time: 2 ms


In [204]:
len(merge_result)

153

time: 2.02 ms


In [253]:
def decode_result(result):
    CATEGORY = [
    "Amount",  # 用药剂量
    "Anatomy",  # 部位
    "Disease",  # 疾病名称
    "Drug",  # 药品名称
    "Duration",  # 持续时间
    "Frequency",  # 用药频率
    "Level",  # 程度
    "Method",  # 用药方法
    "Operation",  # 手术
    "Reason",  # 病因
    "SideEff",  # 不良反应
    "Symptom",  # 临床表现
    "Test",  # 检查方法
    "Test_Value",  # 检查指标值
    "Treatment"  # 非药方法
    ]
    # step1: 去除padding
    merge_result = []
    for r,s in zip(result,sent_pre):
        r = r[10:-10].tolist()
        s = s[10:-10].tolist()
        r_not_pad = []
        for idx,each in enumerate(s):
            if each != 0:
                r_not_pad.append(r[idx])
        merge_result += r_not_pad
    
    # step2：取出最大值
    label_result = []
    for label_vec in merge_result:
        label_idx = np.argmax(label_vec)
        label_result.append(label_idx)
    # return label_result
    
    # Step3：得到标注结果
    cate_flag = 0
    start = -1
    end = -1
    ner_result = []
    for idx,cate in enumerate(label_result):
        if cate == 0 and cate_flag != 0:
            ner_result.append((CATEGORY[cate_flag - 1],start,end + 1,sent[start:end+1]))
            start,end = idx,idx
            cate_flag = cate
        if cate != 0:
            if cate_flag == 0: # 某一个词的开始
                start,end = idx,idx
                cate_flag = cate
            elif cate_flag != 0 and cate == cate_flag: # 在某一个词的中间
                end = idx
            elif cate_flag != 0 and cate != cate_flag: # 已经是另外一个词
                ner_result.append((CATEGORY[cate_flag - 1],start,end + 1,sent[start:end+1]))
                start,end = idx,idx
                cate_flag = cate
    return label_result,ner_result



time: 4 ms


In [254]:
label,ner = decode_result(result)

time: 2 ms


In [256]:
print(label)

[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 3, 3, 3, 0, 9, 9, 9, 9, 9, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0]
time: 1 ms


In [258]:
print(sent)


糖尿病下肢动脉病变多数情况下是全身动脉病变的一部
分,控制动脉硬化的多重危险因素是防治介入治疗后再狭窄的
基础。 糖尿病膝下动脉较膝上动脉更易于发生再狭窄,介入手
术时血管病变越严重,术后再狭窄风险越高,长期糖尿病代谢
异常和 Fg 升高可能是导致再狭窄的原因。

time: 2 ms


In [257]:
ner

[('Disease', 1, 10, '糖尿病下肢动脉病变'),
 ('Disease', 16, 22, '全身动脉病变'),
 ('Disease', 30, 34, '动脉硬化'),
 ('Disease', 49, 52, '再狭窄'),
 ('Disease', 58, 61, '糖尿病'),
 ('Anatomy', 62, 70, '下动脉较膝上动脉'),
 ('Disease', 75, 78, '再狭窄'),
 ('Operation', 79, 84, '介入手\n术'),
 ('Disease', 85, 89, '血管病变'),
 ('Disease', 95, 98, '再狭窄'),
 ('Reason', 103, 113, '长期糖尿病代谢\n异常'),
 ('Reason', 115, 120, 'Fg 升高'),
 ('Disease', 125, 128, '再狭窄')]

time: 3 ms
